In [112]:
import numpy as np
import operator 
import matplotlib.pylab as plt
import os

In [4]:
def createDataSet():
    group = np.array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    label = ['A','A','B','B']
    return group,label

In [35]:
def classify0(inX,dataSet,labels,k):
    dataSetSize = dataSet.shape[0]
    #tile函数是对数据进行复制，这样做可以避免for循环，在数据量大的时候，方便操作。
    diffMat = np.tile(inX,(dataSetSize,1)) - dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sortedDistIndicies = distances.argsort()
    classCount={}
    for i in range(k):
        votelabel = labels[sortedDistIndicies[i]]
        classCount[votelabel] = classCount.get(votelabel,0)+1
    #python3中，用items(),来代替iteritems
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [36]:
group,label = createDataSet()
classify0([0,0],group,label,3)

'B'

In [68]:
def transfer(s):
    result = 0
    if s == "largeDoses":
        result = 1
    if s == "smallDoses":
        result = 2
    if s == "didntLike":
        result = 3
    return result

In [69]:
transfer("didntLike")

3

In [91]:
def file2matrix(filename):
    #读入文本中的数据
    fr = open(filename)
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)
    returnMat = np.zeros((numberOfLines,3))
    classLabelVector = []
    index = 0
    for line in arrayOLines:
        line = line.strip()
        listFromLine = line.split('\t')
        returnMat[index,:] = listFromLine[0:3]
        classLabelVector.append(transfer(listFromLine[-1]))
        index+=1
    return returnMat,classLabelVector

In [92]:
X,y=file2matrix('datingTestSet.txt')

In [93]:
def autoNorm(dataSet):
    #返回每一列元素中的最小值与最大值
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals-minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals,(m,1))
    normDataSet = normDataSet/np.tile(ranges,(m,1))
    return normDataSet,ranges,minVals

In [94]:
norMat,ranges,minvals = autoNorm(X)

In [99]:
def datingClassTest():
    hoRatio = 0.1
    datingDataMat,datingLabels = file2matrix('datingTestSet.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],3)
        print("the classifier came back with:{},the real answer is:{}".format(classifierResult,datingLabels[i]))
        if(classifierResult!=datingLabels[i]):
            errorCount+=1
    print("the total error rate is:{}".format(errorCount/numTestVecs))

In [100]:
datingClassTest()

the classifier came back with:1,the real answer is:1
the classifier came back with:2,the real answer is:2
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:1,the real answer is:1
the classifier came back with:1,the real answer is:1
the classifier came back with:3,the real answer is:3
the classifier came back with:1,the real answer is:1
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:2,the real answer is:2
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:2,the real answe

In [103]:
def classifyPerson():
    resultList = ['not at all','in small doses','in large doses']
    percentTats = float(input("percentage of time spent playing video games?"))
    ffMiles = float(input('frequent flier miles earned per year?'))
    iceCream = float(input('liters of ice cream consumed per year?'))
    datingDataMat,datingLabels = file2matrix('datingTestSet.txt')
    norMat,ranges,minvals = autoNorm(datingDataMat)
    inArr = np.array([ffMiles,percentTats,iceCream])
    classifierResult = classify0((inArr-minvals)/ranges,norMat,datingLabels,3)
    print("You will probably like this person:",resultList[classifierResult-1])

In [104]:
classifyPerson()

percentage of time spent playing video games?10
frequent flier miles earned per year?10000
liters of ice cream consumed per year?0.5
You will probably like this person: in small doses


In [117]:
def img2vector(filename):
    #将图像转换成矩阵
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j]=int(lineStr[j])
    return returnVect

In [115]:
def handwritingClassTest():
    hwLabels = []
    traningFileList = os.listdir('trainingDigits/')
    m = len(traningFileList)
    trainingMat = np.zeros((m,1024))
    for i in range(m):
        fileNameStr = traningFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector('trainingDigits/%s'%fileNameStr)
    testFileList = os.listdir('testDigits/')
    errorCount =0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s'%fileNameStr)
        classfierResult = classify0(vectorUnderTest,trainingMat,hwLabels,3)
        print("the classfier came back with{},the real answer is{}".format(classfierResult,classNumStr))
        if(classfierResult!=classNumStr):
            errorCount+=1
    print("\nthe total error rate is {}".format(errorCount/mTest))

In [116]:
handwritingClassTest()

the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0
the classfier came back with0,the real answer is0


the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with7,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with1,the real answer is1
the classfier came back with2,the real answer is2
the classfier came back with2,the real answer is2
the classfier came back with2,the real answer is2
the classfier came back with2,the real answer is2


the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3
the classfier came back with3,the real answer is3


the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5
the classfier came back with5,the real answer is5


the classfier came back with6,the real answer is6
the classfier came back with6,the real answer is6
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7
the classfier came back with7,the real answer is7


the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
the classfier came back with8,the real answer is8
